# Adaptive Stochastic Gradient Descent

Implementation and comparison of adaptive stochastic gradient descent methods using Python programming language. The efficiency comparison is demonstrated in the logistic regression optimization problem.

## Introduction

**Gradient descent** is one of the most popular numerical methods for solving modern optimization problems: from cost reduction to neural networks training. The thing that all these problems have in common is an aim to describe it as a mathematical model (e.g. mathematical equation) $J(\theta_{i})$, which has a set of independent parameters $\theta_{i}$, and then find specific values of $\theta_{i}$ so $J(\theta_{i})$ will reach the minimum possible value. As an example, in neural network training $J(\theta_{i})$ can be described as a loss between the expected training example and the actual one, where $\theta_{i}$ will be the **weight coefficients** of the neural network. So the goal of the training process is to find the optimal value of **weight coefficients**, so the loss will become minimum and the output value of a neural network will become as close to expected as possible.

By definition, the **gradient** is a vector that points towards the highest  increase of the function value $F$ in high dimensional space $R^{n}$, that can be interpreted as a vector of partial derivatives with respect to each parameter $x_{1}, ..., x_{n}$ of the function $F$ in the specific point $a$:

$\nabla F(a) = \begin{bmatrix}
\frac{\partial f}{\partial x_{1}} (a) \\
\vdots \\
\frac{\partial f}{\partial x_{n}} (a) \\
\end{bmatrix}$

Various optimization algorithms, which are based on **gradient descent**, have common feature: usage of a vector, opposite to the gradient value (antigradient or $- \nabla F$), as the key direction to the function minimum. In this research will be overviewed the following algorithms:
- SGD
- Nesterov AG
- ADAGRAD
- RMSPROP
- ADAM
- Dynamic Sampling SGD
- Mini-batch SGD

## Tensorflow API usage

In this overview, the following gradient algorithms will be implemented using TensorFlow API to speed up the process of gradient calculation in the neural language processing problem using [Calculation Graph](https://www.tensorflow.org/api_docs/python/tf/Graph). The gradient value for the following methods will be calculated using [tf.GradientTape()](https://www.tensorflow.org/api_docs/python/tf/GradientTape) in order to achieve higher performance and calculation parallelism. The following code demonstrates class usage in the optimization problem for the **Rosenbrock function**:

$F(x, y) = (1 - x)^{2} + 100(y - x^{2})^{2}$

Function has a single local minimum in $(x, y) = (1, 1)$ and is equal $F(x, y) = 0$ in the following point.

The gradient value, by definition, is a vector that points towards the direction of the greatest increase of the function. In order to reach a local minimum using an iterative algorithm, the next step should be in the direction opposite to the gradient value. So the full algorithm of finding local minimum can be described as $x_{i+1} = x_{i} - \lambda_{i}\nabla F$, where $\lambda_{i} = const$ - size of a single step, $x_{i}$ - current step.

In [ ]:
import tensorflow as tf

In [ ]:
# Single iteration of an algorithm
def gradient_step(alpha: tf.constant, x: tf.Variable, y: tf.Variable) -> tf.Variable:
  with tf.GradientTape() as grad:
    grad.watch((alpha, x, y))

    # Optimization function
    F = (1.0 - x) ** 2.0 + 100.0 * (y - x ** 2.0) ** 2.0
    [dF_dx, dF_dy] = grad.gradient(F, [x, y])

  return x - alpha * tf.Variable(dF_dx), y - alpha * tf.Variable(dF_dy)

# Gradient steps
steps = 1000

# Gradient step size
alpha = tf.constant(0.001)

# Start conditions
x = tf.Variable(2.0)
y = tf.Variable(-1.0)

# Iterations
xi = []; yi = []

for _ in range(steps):
  # Iterative descent
  x, y = gradient_step(alpha, x, y)
  xi.append(x.numpy()); yi.append(y.numpy())

print(f"Closest approximation of the local minimum: [{xi[-1]}, {yi[-1]}]")

Closest approximation of the local minimum: [0.9669684171676636, 0.9348930716514587]


Using [matplotlib](https://matplotlib.org/) library, all recorded steps $\{ x_{1}, x_{n} \}$ can be visualised in a following 3D plot.

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.cm import coolwarm as cmap
from numpy import arange, meshgrid, array
from ipywidgets import interact, IntSlider

Visualisation intervals and rendering density

In [ ]:
density = 0.1

X = arange(-2.0, 2.0, density)
Y = arange(-1.0, 3.0, density)

X, Y = meshgrid(X, Y)

**Rosenbrock function** in vector form:

In [ ]:
F = lambda x, y: (1.0 - x) ** 2.0 + 100.0 * (y - x ** 2.0) ** 2.0

Descent visualisation for space $(x, y) \in \{ -2 \leq x \leq 2; -1 \leq y \leq 3 \}$

In [ ]:
@interact(elev=IntSlider(min=0, max=360, step=1, value=30),
          azim=IntSlider(min=0, max=360, step=1, value=50))
def plot_descent(elev: int, azim: int):
  # Descent steps plotting
  xs = array(xi).flatten()
  ys = array(yi).flatten()

  fs = F(xs, ys).flatten()

  # Meshgrid plotting
  fig = plt.figure()
  ax = fig.gca(projection='3d')
  ax.view_init(elev=elev, azim=azim)

  ax.plot(xs, ys, fs, lw=0.5, marker='*', color='black')
  surf = ax.plot_surface(X, Y, F(X, Y), cmap=cmap, antialiased=True)
  fig.colorbar(surf)

  plt.show()

interactive(children=(IntSlider(value=30, description='elev', max=360), IntSlider(value=50, description='azim'…

## Attribution

Overview is based on research paper "An overview of gradient descent optimization algorithms" from [arXiv.org](https://arxiv.org/pdf/1609.04747.pdf) by [Sebastian Ruder](mailto:ruder.sebastian@gmail.com) licensed under CC BY-NC-SA 4.0.